In [ ]:
import sys
from pathlib import Path

In [ ]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

2022-11-25 19:26:52.107515: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-25 19:26:52.246160: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-25 19:26:52.758067: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64
2022-11-25 19:26:52.758147: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: can

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
sys.path.append(str(Path("../../ssl").resolve()))

In [ ]:
from src.models.pi_model.pi_model import PiModel
from src.models.pi_model.pi_model_config import PiModelConfig
from src.trainers.meta_pseudo_label.meta_pseudo_label import MetaPseudoLabelTrainer
from src.trainers.meta_pseudo_label.meta_pseudo_label_config import MetaPseudoLabelTrainerConfig
from src.data_loaders.meta_pseudo_label.meta_pseudo_label_config import MetaPseudoLabelDataLoaderConfig
from src.data_loaders.meta_pseudo_label.meta_pseudo_label import MetaPseudoLabelDataLoader

# Description

In this notebook, the AlexNet model will be trained on the CIFAR-10 dataset using only 25% of the labelled dataset.

## Set up Experiment

In [ ]:
class TrainerConfig(MetaPseudoLabelTrainerConfig):
    num_epochs = 200
    uda_loss_weight = 1.0

train_config = TrainerConfig()

In [ ]:
class ModelConfig(PiModelConfig):
    input_shape = (32, 32, 3)
    output_shape = 10

model_config = ModelConfig()

In [ ]:
class DataLoaderConfig(MetaPseudoLabelDataLoaderConfig):
    batch_size = 64
    num_classes = 10
    shuffle_buffer_size = 50000 # dataset size    

    # apply augmentations
    blur_params = {
        'chance': 0.10,
        'kernel_ratio': 0.10,
        'blur_strength': (0.1, 2.0)
    }

    crop_params = {
        'chance': 0.50,
        'crop_size': (0.08, 1.0),
        'aspect_range': (0.75, 1.33),
        'num_tries': 100
    }

    jitter_params = {
        'chance': 0.50,
        'distort_strength': 0.20,
        'drop_chance': 0.05
    }

data_loader_config = DataLoaderConfig()

## Get Datasets

In [ ]:
(x_train_full, y_train_full), (x_test_full, y_test_full) = tf.keras.datasets.cifar10.load_data()

In [ ]:
# get train dataset by subsampling 10% of the full training dataset (stratified by labels)
# add the rest as unlabelled samples
x_train_unlabelled, x_train_labelled, y_train_unlabelled_, y_train_labelled = train_test_split(
    x_train_full, y_train_full,
    stratify = y_train_full,
    test_size = 0.10, random_state = 42
)

# cast types
y_train_unlabelled = (-1 * np.ones_like(y_train_unlabelled_)).astype(np.int64)
y_train_labelled = y_train_labelled.astype(np.int64)
y_test_full = y_test_full.astype(np.int64)

In [ ]:
x_train = np.concatenate((x_train_unlabelled, x_train_labelled), axis = 0)
y_train = np.concatenate((y_train_unlabelled, y_train_labelled), axis = 0)

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_data = tf.data.Dataset.from_tensor_slices((x_test_full, y_test_full))

2022-11-25 19:26:55.225544: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-25 19:26:55.232218: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-25 19:26:55.232748: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-25 19:26:55.233722: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [ ]:
# create train dataset
train_data = MetaPseudoLabelDataLoader(train_data, data_loader_config)(training = True)

In [ ]:
# create test dataset
val_data = MetaPseudoLabelDataLoader(val_data, data_loader_config)(training = False)

In [ ]:
print(f"Train dataset size: {train_data.cardinality()}")
print(f"Validation dataset size: {val_data.cardinality()}")

# Train Model

In [ ]:
model_student = PiModel(model_config)()
model_teacher = PiModel(model_config)()

In [ ]:
trainer = MetaPseudoLabelTrainer(
    model_student,
    model_teacher,
    train_data,
    train_config,
    val_dataset = val_data)

In [ ]:
trainer.train()

2022-11-25 19:26:58.147850: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204


Training loss at epoch 0 is : ltsu 2.14, ltss 2.82, lttu -0.11, ltts 2.12, lttuda 0.00. Validation loss is : 3.71. Validation acc. is : 10.00.
Training loss at epoch 1 is : ltsu 2.16, ltss 2.52, lttu -0.03, ltts 1.91, lttuda 0.02. Validation loss is : 2.40. Validation acc. is : 10.00.
Training loss at epoch 2 is : ltsu 1.86, ltss 2.54, lttu 0.22, ltts 1.82, lttuda 0.02. Validation loss is : 2.25. Validation acc. is : 25.94.
Training loss at epoch 3 is : ltsu 1.50, ltss 2.34, lttu 0.26, ltts 1.76, lttuda 0.03. Validation loss is : 2.13. Validation acc. is : 32.75.
Training loss at epoch 4 is : ltsu 1.43, ltss 2.18, lttu 0.29, ltts 1.71, lttuda 0.03. Validation loss is : 1.97. Validation acc. is : 34.84.
Training loss at epoch 5 is : ltsu 1.37, ltss 2.10, lttu 0.25, ltts 1.66, lttuda 0.04. Validation loss is : 1.81. Validation acc. is : 38.47.
Training loss at epoch 6 is : ltsu 1.36, ltss 2.01, lttu 0.26, ltts 1.60, lttuda 0.05. Validation loss is : 1.89. Validation acc. is : 39.38.
Trai